In [17]:
import tensorflow as tf
import keras

In [18]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'

In [19]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 64
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001


In [20]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    classes=['normal','COVID-19'],
    class_mode='binary',
    subset='training'
)

Found 5805 images belonging to 2 classes.


In [21]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal','COVID-19'],
    subset='training'
)

Found 905 images belonging to 2 classes.


In [22]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal','COVID-19'],
        #subset='validation'
)

Found 905 images belonging to 2 classes.


In [23]:
pretrained_model = keras.models.load_model("F:/Xray_Gaus/CNN_binary_wt")


In [24]:
batch_size = 64

epochs=5

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "tl_med_cnn.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=1e-04                 #0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]


In [30]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=pretrained_model.fit(x = train_gen,
          batch_size=batch_size,
          epochs=2,
          callbacks=callbacks,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID )

Epoch 1/2
90/90 [==============================] - 261s 3s/step - loss: 0.0311 - accuracy: 0.9915 - val_loss: 12.3492 - val_accuracy: 0.0346
Epoch 2/2
90/90 [==============================] - 308s 3s/step - loss: 0.0196 - accuracy: 0.9932 - val_loss: 0.3869 - val_accuracy: 0.8717


In [31]:
results = pretrained_model.evaluate(test_gen, batch_size=64)

29/29 [==============================] - 22s 765ms/step - loss: 0.0314 - accuracy: 0.9878


In [32]:

y_pred = pretrained_model.predict(test_gen, verbose=1)
#y_pred = np.argmax(y_pred, axis = 1)

#y_pred=model.predict(X_test)

for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

29/29 [==============================] - 15s 534ms/step


In [33]:
y_true=test_gen.labels

In [34]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.98      0.97      0.98       885
           1       0.07      0.10      0.09        20

    accuracy                           0.95       905
   macro avg       0.53      0.54      0.53       905
weighted avg       0.96      0.95      0.96       905



In [36]:
pretrained_model.save('F:/Xray_Gaus/tlCNN_binary_wt')

history.history

INFO:tensorflow:Assets written to: F:/Xray_Gaus/tlCNN_binary_wt\assets


{'loss': [0.03113209269940853, 0.01964982971549034],
 'accuracy': [0.9914649128913879, 0.9932067394256592],
 'val_loss': [12.349210739135742, 0.3869269788265228],
 'val_accuracy': [0.0345982126891613, 0.8716517686843872],
 'lr': [0.001, 0.001]}